# Projet Datascientest | Rapport technique

**Participants** : Olga TOLSTOLUTSKA, Mohamed BACHKAT, Charly LAGRESLE \
**Mentor** : Manu PORTEL \
**Promotion** : DST Bootcamp DEC22 \
**Date du document** : 2023/03/06



## Résumé





# Introduction

## Contexte et objectifs

### Etapes

Rakuten est un site de e-commerce, regroupant près de 1.3 milliards d'utilisateurs. Afin de faciliter la gestion des suggestions de leurs recherches ou des recommandations qui leurs seront proposées, il est important de classifier les produits. Du fait de la quantité de produits et de leurs catégories associées, ce classement ne peut ni être fait manuellement, ni être régi par des règles. Une approche plus flexible, efficace et reproductible est donc à mettre en place. 

### Objectif

L'objectif est de prédire le code de chaque produit, c'est-à-dire sa catégorie, sur la base d'une analyse de données textuelles et d'images.

### Etapes du projet

Le déroulement du projet suit la statégie suivante :

* Analyse exploratoire
    * Prise en main des données
    * Analyse statistique et feature-engineering
* Conception de modèles de référence en machine learning
* Conception de modèles deep-learning
    * Traitement des images et du texte
    * Fusion de modèles
* Développement d'un outil intercatif

Le rapport ici présent se compose de parties similaires.

# Analyse exploratoire

L'exploration des données est la première étape de ce projet. 

## Prise en main des données

Les données fournies pour ce challenge sont les suivantes :
* `X_train_update.csv` : données explicatives et textuelles pour l'entraînement
* `X_test_update.csv` : données explicatives et textuelles pour les tests
* `Y_train_CVw08PX.csv` : variable cible pour l'entraînement
* `images/image_train/*.jpg` : images pour l'entraînement
* `images/image_test/*.jpg` :  images pour les tests

> Nous notons que la varible cible de test n'est pas fournie : les observations de tests ne seront donc pas exploitées.

Le jeu d'entrainement se compose a priori de 84 916 observations : des données textuelles ainsi que des images.

### Données d'entrées



#### Textes

Les données `X_train_update.csv` se compose de plusieurs champs : 

* `designation` : titre du produit dans différentes langues (0% de NaNs)
* `description` : description détaillée du produit dans différentes langues (35% de NaNs)
* `productid` : identifiant unique du produit (0% NaNs)
* `imageid` identifiant unique de l'image du produit (0% NaNs)

> Les champs `productid` et `imageid` ne servent qu'à construire le nom du fichier image associé au produit. Ils ne seront donc pas étudiés statistiquement parlant.

#### Images

Chaque produit est associé à une image qui le représente. L'image comporte le produit, généralement photographié sur un fond blanc, mais peut également être complété par une mise scène du produit (jardin, décoration) ou par un support (main portant le produit par exemple). Les images sont en couleur, ont une taille de 500x500 pixels et nous sont fournies au format `.jpg`.

### Données de sorties

Les cibles sont fournies dans le fichier `Y_train_CVw08PX.csv`. A ce stade, nous notons que 27 catégories différentes seront à déterminer.


### Analyse du problème

#### Type de problème

Les observations textuelles et graphiques doivent permettre au modèle d'affecter à chaque observation dont les features sont actuellement (`designation`, `description` , et une image) à une catégorie de produit `prdtypecode`. Cette cible est disponible et se présente sous la forme d'un numéro associé à une catégorie (par exemple `livre`, `tech`, `décoration`, etc.).

Nous avons donc affaire à un problème de catégorisation supervisé. Pour la partie texte, un analyse automatique du langage naturel (*NLP : Natural Langage Processing*) sera à mettre en place.

#### Particularité du problème

La catégorisation se base sur l'apprentissage fait sur les données à notre disposition. Ces données sont de deux types (textuel et graphique). De ce fait :
* les pré-traitements seront différents
* plusieurs modèles seront à concevoir, à entrainer et à faire dialoguer
    * un modèle pour catégoriser le produit en fonction du texte fourni
    * un modèle pour catégrosier le produit en fonction de son image associée
    * une étape de regroupement des prédictions sera à réaliser afin d'obtenir une unique catégorie en fin de process


## Analayse statistique et feature-engineering

### Données d'entrée : textes

#### Nombre de mots

Pour `designation`, les textes sont composés de 4 à 54 mots. La répartition est de type gaussienne : la moyenne se situe à 11.5 mots et l'écart-type est de 6.4 mots. 75% des textes de `designation` ont moins de 14 mots. 

Pour `description`, les textes plus longs et comportent entre 0 (certaines descriptions sont vides) et 2 068 mots. En moyenne, la description se compose d'un texte de 78 mots et 75% des produits sont décrits avec moins de 124 mots. La répartition du nombre de mots suit ici la densité de probabilité d'une loi exponentielle.

TODO  : afficher graphique

Plusieurs actions seront mises en place suite à cette étude

* Les colonnes `designation` et `description` seront fusionnées en une colonne `text`
* La colonne `text` sera limitée à 500 mots.


#### Langues

Intéressons-nous maintenant à la langue. Les colonnes `designation` et `description` contiennent du texte dont la langue peut varier. Le graphique ci-dessous montre la répartition des langues pour la colonne `designation`. La répartition des langues est très hétérogène :
* 81% en français
* 14% en anglais
* 1.5% en allemand
* inférieurs à 1.5% : nl, ca, it, ro, pt, etc...

Le choix a été fait de traduire les textes étrangers vers le français.

TODO : affichger le graphique

### Données d'entrée : images

#### Réduction de dimension

Nous avons noté que beaucoup d'images sont photographiées sur un fond blanc. Ce fond n'apporte en soi aucune information et peut mener à un biais dans notre modèle de classification des images.

TODO : afficher masque et distribution

En faisant une analyse de la variance des pixels sur un échantillon d'images, nous notons que les bordures des images présentent une variance très basses. 

Les images seront rognées de 20% pour supprimer ces marges porteuses de peu d'information avant d'être redimensionnées à la dimension souhaitée.

### Données de sorties

Commme expliqué précédemment, 27 catégories sont présentes dans notre dataset. 

#### Déséquilibre

Le graphique ci-dessous montre le nombre de produits pour chacune des 27 catégories. On remarque facilement que la catégorie "Aquatique" est sur-représentée : elle comporte le double de produits par rapport à la catégorie suivante. On observe également une baisse linéaire du nombre de produits de `Affiche` vers `Accessoire console`. Les catégories de `Cuisine` à `Jeu Console 2` ne présentent que peu d'individus. 

TODO : afficher le graphique

#### Catégories

En visualisant chacune de ces catégories, nous pouvons traduire le numéro de la catégorie à un type de produit.

|Numéro de catégorie| Description|
|-------------------|-------------|
| 10 | Libre |
| 40 | Jeu Console |
| 50 | Accessoire Console|
| 60 | Tech |
| 1140 | Figurine |
| 1160 | Carte Collection |
| 1180 | Jeu Plateau |
| 1280 | Déguisement |
| 1281 | Boite de jeu |
| 1300 | Jouet Tech |
| 1301 | Chaussette |
| 1302 | Gadget |
| 1320 | Bébé |
| 1560 | Salon |
| 1920 | Chambre |
| 1940 | Cuisine |
| 2220 | Jeu Manuel |
| 2060 | Chambre enfant |
| 2220 | Animaux |
| 2280 | Affiche |
| 2403 | Vintage |
| 2462 | Jeu oldschool |
| 2522 | Bureautique |
| 2582 | Décoration |
| 2583 | Aquatique |
| 2585 | Soin et Bricolage |
| 2705 | Livre |
| 2905 | Jeu Console 2 |




# Modèles de références

# Modèles de deep-learning

## Traitement des images et du texte

## Fusion des modèles

# Outil interactif

# Conclusion

